In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/Computer Vision-Final Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Computer Vision-Final Project


# **Load Drone Dataset**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
#
# Drone Dataset:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
class DroneDataset(Dataset):

    def __init__(self, img_path, mask_path, X, mean, std, transform=None):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        
        img = cv2.imread(self.img_path + '/'+self.X[i] + '.jpg')
        print(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path +'/'+ self.X[i] + '.png', cv2.IMREAD_GRAYSCALE)

        # how to transform?
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = Image.fromarray(augmented['image'])
            mask = augmented['mask']
        else:
            img = Image.fromarray(img)
        trans = T.Compose(
            [T.ToTensor(),
             T.Normalize(self.mean, self.std)]
        )
        img = trans(img)
        # simply calling the below will not work, you have to forward it to return something,
        # thus, we have to use T.Compose and then call it.
        # img = T.ToTensor()
        # img = T.Normalize(self.mean, self.std)
        mask = torch.from_numpy(mask).long()

        # if self.patches:
        #     img, mask = self.tiles(img, mask)

        return img, mask


class DroneTestDataset(Dataset):

    def __init__(self, img_path, mask_path, X):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + '0' + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)

        mask = torch.from_numpy(mask).long()

        return img, mask


In [ ]:
import os
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torch.nn.functional as F



from sklearn.model_selection import train_test_split

from PIL import Image
import cv2


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

IMAGE_PATH = 'drone_dataset/images'
MASK_PATH = 'drone_dataset/dataset/label_processed'


# create df with id of the dataset
def create_df(path):
    name = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            name.append(filename.split('.')[0])

    return pd.DataFrame({'id': name}, index=np.arange(0, len(name)))


df = create_df(IMAGE_PATH)
print('Total Images: ', len(df))



# # split the dataset into train, validation and test data
X_trainval, X_test = train_test_split(df['id'].values, test_size=0.1, random_state=0)
X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=0)


# Define mean and std value
# Drone Dataset
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# create datasets
train_set = DroneDataset(IMAGE_PATH, MASK_PATH, X_train, mean, std)
val_set = DroneDataset(IMAGE_PATH, MASK_PATH, X_val, mean, std)
# load data--->define batch size
batch_size = 3

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)


Total Images:  400
[[[111 113 107]
  [110 112 106]
  [110 112 106]
  ...
  [ 65  58  55]
  [ 67  60  57]
  [ 68  61  58]]

 [[111 113 107]
  [111 113 107]
  [110 112 106]
  ...
  [ 72  65  62]
  [ 70  63  60]
  [ 68  61  58]]

 [[112 114 108]
  [111 113 107]
  [111 113 107]
  ...
  [ 74  67  64]
  [ 73  66  63]
  [ 69  62  59]]

 ...

 [[102 104  98]
  [103 105  99]
  [105 107 101]
  ...
  [ 61  57  52]
  [ 62  58  53]
  [ 63  59  54]]

 [[101 103  97]
  [102 104  98]
  [104 106 100]
  ...
  [ 62  58  53]
  [ 63  59  54]
  [ 65  61  56]]

 [[101 103  97]
  [102 104  98]
  [104 106 100]
  ...
  [ 61  57  52]
  [ 64  60  55]
  [ 65  61  56]]]
(tensor([[[-0.2856, -0.3027, -0.3027,  ..., -1.1760, -1.1418, -1.1247],
         [-0.2856, -0.2856, -0.3027,  ..., -1.0562, -1.0904, -1.1247],
         [-0.2684, -0.2856, -0.2856,  ..., -1.0219, -1.0390, -1.1075],
         ...,
         [-0.4397, -0.4226, -0.3883,  ..., -1.2274, -1.2103, -1.1932],
         [-0.4568, -0.4397, -0.4054,  ..., -1.2103, 

# **Resize**

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


image_path = "drone_dataset/images"



def loadImages(path):
    # Put file names into lists
    image_files = sorted([os.path.join(path,file)
         for file in os.listdir(path)])
 
    return image_files

image_files = loadImages(image_path)
print(image_files)
print(len(image_files)) #400, good.


# TEST ============================================================================
# test the first picture

# test_img = Image.open(image_files[0])
# test_img = test_img.resize((750, 500),Image.ANTIALIAS)
# # print(test_img)

# output_filename = image_files[0][-7:]
# test_img.save(os.path.join(image_path,'compressed_img', output_filename))
# TEST END ============================================================================


def processImages(image_files,image_path):
    for i, item in enumerate(image_files):
        print("Processing:", i, item)
        img = Image.open(item)
        img = img.resize((768, 512),Image.ANTIALIAS)
        # print(test_img)

        output_filename = item[-7:]
        # img.save(os.path.join(image_path,'compressed_img', output_filename))
        img.save(os.path.join(image_path,output_filename))

processImages(image_files)

['drone_dataset/images/000.jpg', 'drone_dataset/images/001.jpg', 'drone_dataset/images/002.jpg', 'drone_dataset/images/003.jpg', 'drone_dataset/images/004.jpg', 'drone_dataset/images/005.jpg', 'drone_dataset/images/006.jpg', 'drone_dataset/images/008.jpg', 'drone_dataset/images/011.jpg', 'drone_dataset/images/013.jpg', 'drone_dataset/images/014.jpg', 'drone_dataset/images/015.jpg', 'drone_dataset/images/016.jpg', 'drone_dataset/images/018.jpg', 'drone_dataset/images/019.jpg', 'drone_dataset/images/021.jpg', 'drone_dataset/images/022.jpg', 'drone_dataset/images/023.jpg', 'drone_dataset/images/026.jpg', 'drone_dataset/images/028.jpg', 'drone_dataset/images/031.jpg', 'drone_dataset/images/035.jpg', 'drone_dataset/images/038.jpg', 'drone_dataset/images/040.jpg', 'drone_dataset/images/041.jpg', 'drone_dataset/images/042.jpg', 'drone_dataset/images/043.jpg', 'drone_dataset/images/044.jpg', 'drone_dataset/images/045.jpg', 'drone_dataset/images/047.jpg', 'drone_dataset/images/049.jpg', 'drone_

In [ ]:
image_path = 'drone_dataset/dataset/label_processed'

image_files = loadImages(image_path)
print(len(image_files)) 
def processImages(image_files,image_path):
    for i, item in enumerate(image_files):
        print("Processing:", i, item)
        img = Image.open(item)
        img = img.resize((768, 512),Image.ANTIALIAS)
        # print(test_img)

        output_filename = item[-7:]
        # img.save(os.path.join(image_path,'compressed_img', output_filename))
        img.save(os.path.join(image_path,output_filename))
processImages(image_files,image_path)

400
Processing: 0 drone_dataset/dataset/label_processed/000.png
Processing: 1 drone_dataset/dataset/label_processed/001.png
Processing: 2 drone_dataset/dataset/label_processed/002.png
Processing: 3 drone_dataset/dataset/label_processed/003.png
Processing: 4 drone_dataset/dataset/label_processed/004.png
Processing: 5 drone_dataset/dataset/label_processed/005.png
Processing: 6 drone_dataset/dataset/label_processed/006.png
Processing: 7 drone_dataset/dataset/label_processed/008.png
Processing: 8 drone_dataset/dataset/label_processed/011.png
Processing: 9 drone_dataset/dataset/label_processed/013.png
Processing: 10 drone_dataset/dataset/label_processed/014.png
Processing: 11 drone_dataset/dataset/label_processed/015.png
Processing: 12 drone_dataset/dataset/label_processed/016.png
Processing: 13 drone_dataset/dataset/label_processed/018.png
Processing: 14 drone_dataset/dataset/label_processed/019.png
Processing: 15 drone_dataset/dataset/label_processed/021.png
Processing: 16 drone_dataset/d

# **Load GTA5**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
#
# GTA5 Dataset:
mean = [0.28689554, 0.32513303, 0.28389177]
std = [0.18696375, 0.19017339, 0.18720214]
class GTA5(Dataset):

    def __init__(self, img_path, mask_path, X, mean, std, transform=None):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        img = cv2.imread(self.img_path + self.X[i] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + '0' + self.X[i] + '.png', cv2.IMREAD_GRAYSCALE)

        # how to transform?
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = Image.fromarray(augmented['image'])
            mask = augmented['mask']
        else:
            img = Image.fromarray(img)
        trans = T.Compose(
            [T.ToTensor(),
             T.Normalize(self.mean, self.std)]
        )
        img = trans(img)
        mask = torch.from_numpy(mask).long()


        return img, mask


class GTA5_test(Dataset):

    def __init__(self, img_path, mask_path, X):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + '0' + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)

        mask = torch.from_numpy(mask).long()

        return img, mask

In [ ]:
import os
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torch.nn.functional as F



from sklearn.model_selection import train_test_split

from PIL import Image
import cv2


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

IMAGE_PATH = 'GTA5/dataset/images'
MASK_PATH = 'GTA5/dataset/label_processed'


# create df with id of the dataset
def create_df(path):
    name = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            name.append(filename.split('.')[0])

    return pd.DataFrame({'id': name}, index=np.arange(0, len(name)))


df = create_df(IMAGE_PATH)
print('Total Images: ', len(df))



# split the dataset into train, validation and test data
X_trainval, X_test = train_test_split(df['id'].values, test_size=0.1, random_state=0)
X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=0)


# Define mean and std value
mean = [0.28689554, 0.32513303, 0.28389177]
std = [0.18696375, 0.19017339, 0.18720214]

# create datasets
train_set = GTA5(IMAGE_PATH, MASK_PATH, X_train, mean, std)
val_set = GTA5(IMAGE_PATH, MASK_PATH, X_val, mean, std)
# load data--->define batch size
batch_size = 3

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

Total Images:  4
